<img src="Images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

## Exercise: Compute Median Temperature

In many cases, porting code from CPU to GPU is as simple as replacing `std::` with `thrust::`.
To verify this, we'll focus on calculating the median temperature in the vector. 
You'll start with a CPU-based implementation and modify the code to run on the GPU. 
Below is the original CPU code that you'll need to adapt:

In [ ]:
#@title Google Colab Setup
!mkdir -p Sources
!wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/gpu-cpp-tutorial/notebooks/01.02-Execution-Spaces/Sources/ach.h -nv -O Sources/ach.h

2025-02-10 23:45:15 URL:https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/gpu-cpp-tutorial/notebooks/01.02-Execution-Spaces/Sources/ach.h [2893/2893] -> "Sources/ach.h" [1]


In [ ]:
%%writefile Sources/port-sort-to-gpu.cpp
#include "ach.h"

float median(thrust::universal_vector<float> vec) 
{
    std::sort(vec.begin(), vec.end());
    return vec[vec.size() / 2];
}

int main() 
{
    float k = 0.5;
    float ambient_temp = 20;
    thrust::universal_vector<float> temp{ 42, 24, 50 };
    auto transformation = [=] __host__ __device__ (float temp) { return temp + k * (ambient_temp - temp); };

    std::printf("step  median\n");
    for (int step = 0; step < 3; step++) {
        thrust::transform(thrust::device, temp.begin(), temp.end(), temp.begin(), transformation);
        float median_temp = median(temp);
        std::printf("%d     %.2f\n", step, median_temp);
    }
}

In [ ]:
!nvcc -o /tmp/a.out --extended-lambda Sources/port-sort-to-gpu.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

nvcc warning : Support for offline compilation for architectures prior to '<compute/sm/lto>_75' will be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
step  median
0     31.00
1     25.50
2     22.75


If everything goes well, the cell above should print:

| step | median
| :--- | :-----
| 0    | 31.00
| 1    | 25.50
| 2    | 22.75

If you’re unsure how to proceed, consider expanding this section for guidance. Use the hint only after giving the problem a genuine attempt.

<details>
  <summary>Hints</summary>
  
  - Thrust provides a `thrust::sort` function that can be used to sort data on the GPU
  - Use `thrust::device` execution policy to specify where you want `thrust::sort` to run
</details>

Open this section only after you’ve made a serious attempt at solving the problem. Once you’ve completed your solution, compare it with the reference provided here to evaluate your approach and identify any potential improvements.

<details>
  <summary>Solution</summary>

  Key points:
  - change `std::sort` to `thrust::sort`
  - add `thrust::device` execution policy parameter

  Solution:
  ```c++
  float median(thrust::universal_vector<float> vec) {
    thrust::sort(thrust::device, vec.begin(), vec.end());
    return vec[vec.size() / 2];
  }
  ```

  You can find full solution [here](Solutions/port-sort-to-gpu.cu).
</details>


---

Now you should be comfortable with the concept of execution spaces and be able to port standard algorithms to GPU with Thrust!

But what if your algorithm is not a standard one?

Proceed to the [next section](../01.03-Extending-Algorithms/01.03.01-Extending-Algorithms.ipynb) to learn how to extend standard algorithms to your unique use cases.

<img src="Images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">